In [ ]:
#Import ultralytics
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 23.3/107.7 GB disk)


In [ ]:
# Import required libraries
import os
import shutil
import random

#!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
# Connect to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Training the model

## Creating train-val split

In [ ]:
#Make new file pats for splitting training data
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.25):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(101)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')



    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/UU_Thesis/yolov8/_Training_data_/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 55 images---


  0%|          | 0/42 [00:00<?, ?it/s]

------ Training data created with 80% split 42 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

------ Testing data created with a total of 13 images ----------
------ TASK COMPLETED -------


## Training

In [ ]:
#Train model - with X epochs and X batches
!yolo task=detect mode=train model=yolov8m.pt data=/content/drive/MyDrive/UU_Thesis/yolov8/dataset.yaml  epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/UU_Thesis/yolov8/training_results name=goats13_6_paint_grande2

100% 49.7M/49.7M [00:00<00:00, 315MB/s]
Ultralytics YOLOv8.0.117 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/UU_Thesis/yolov8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/UU_Thesis/yolov8/training_results, name=goats13_6_paint_grande2, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, c

## Executing model

In [ ]:
#Assign model and test images
from ultralytics import YOLO
model=YOLO('/content/drive/MyDrive/UU_Thesis/yolov8/training_results/goats13_6_paint_grande24/weights/best.pt')
source='/content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/selected_images'
print(type(model))

<class 'ultralytics.yolo.engine.model.YOLO'>


In [ ]:
#Predict images
results = model.predict(source, save=True, conf=0.15)


image 1/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1000.JPG: 640x480 2 Goats, 453.7ms
image 2/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1003.jpg: 640x480 2 Goats, 541.1ms
image 3/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1005.jpg: 480x640 2 Goats, 459.1ms
image 4/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1010.JPG: 480x640 2 Goats, 468.3ms
image 5/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1019.jpg: 480x640 1 Goat, 451.8ms
image 6/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1024.jpg: 448x640 4 Goats, 448.4ms
image 7/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1030.jpg: 416x640 1 Goat, 377.1ms
image 8/135 /content/drive/.shortcut-targets-by-i

# Storing results

In [ ]:
# Set the directory where the file will be saved
directory = "/content/drive/MyDrive/UU_Thesis/yolov8/output/output_selected_grande_save.15"

os.makedirs(directory)

for n, result in enumerate(results):
    boxes = str(result.boxes.xywh)     #xyxy format /or xywh format
    #print("For result ", n, result,"data =",boxes)
    #print("Boxes Conf = ", result.boxes.conf)

    #Get image path
    path = result.path
    filename = path[path.rfind("/") + 1:]

    new_extension = '.txt'

    if '.' in filename:
        # Split the filename into the base name and extension
        parts = filename.rsplit('.', 1)

        # Replace the extension with the new extension
        new_filename = f"{parts[0]}{new_extension}"
    else:
        # If the filename doesn't have an extension, simply append the new extension to the end
        new_filename = f"{filename}{new_extension}"

    #print(new_filename)

    # Construct the full path to the new file
    new_file_path = os.path.join(directory, new_filename)

    # Open the new file for writing
    with open(new_file_path, "w") as f:
        # Write the contents of 'boxes' to the file
        f.write(boxes)

############### Final detection done in CombinedNotebook_incl_segment.ipynb

In [ ]:
!yolo task=detect mode=predict model='/content/drive/MyDrive/UU_Thesis/yolov8/training_results/goats_paint_grande/weights/best.pt' conf=0.25 source='/content/drive/MyDrive/UU_Thesis/yolov8/_Test_data_/selected_images'

Ultralytics YOLOv8.0.103 🚀 Python-3.10.11 torch-2.0.0+cu118 CPU
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1000.JPG: 640x480 (no detections), 911.7ms
image 2/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1003.jpg: 640x480 2 Goats, 952.8ms
image 3/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1005.jpg: 480x640 2 Goats, 827.4ms
image 4/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1010.JPG: 480x640 1 Goat, 776.1ms
image 5/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1019.jpg: 480x640 1 Goat, 758.9ms
image 6/135 /content/drive/.shortcut-targets-by-id/1KtvCG64y-41VDxfO0qDSr53zjhVQ7b-I/selected_images/1024.jpg: 448x640 3 Goats, 688.5ms
image 7/135 /content/drive/.shor

In [ ]:
!cp -r /content/runs/detect/predict '/content/drive/MyDrive/UU_Thesis/yolov8/output'

save=True